<a href="https://colab.research.google.com/github/mikeogunmakin/river-medway-trading/blob/main/research/202511_automate_brier_score_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Automate Brier Score Analysis**

**Aim:** Automate brier score analysis to quantify the predictability of a football league.

Pseudo code logic:

Function takes a dataset as input and returns the Brier score for Home, Draw, and Win markets.

1. Create binary outcome columns for each market (Home, Draw, Win).
2. Calculate the implied probabilities from the market odds:
   - If Betfair Exchange market odds are unavailable, standardise and use the average odds instead.
3. For each market (Home, Draw, Win), calculate the squared error between the implied probabilities and actual outcomes.
4. Compute the mean of the squared errors for each market to obtain the Brier score.
5. Return the Brier score for each market.